In [15]:
import pandas as pd

# Load the dataset
file_path = 'C:\!Projects\DI-Bootcamp\Week9\Day2\OnlineRetail.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

<>:4: SyntaxWarning: invalid escape sequence '\!'
<>:4: SyntaxWarning: invalid escape sequence '\!'
C:\Users\kyana\AppData\Local\Temp\ipykernel_16532\3624717101.py:4: SyntaxWarning: invalid escape sequence '\!'
  file_path = 'C:\!Projects\DI-Bootcamp\Week9\Day2\OnlineRetail.csv'


In [16]:
# Convert 'InvoiceDate' to datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [17]:
# Using Gemini OpenAI
import google.generativeai as genai
import apikey

key = apikey.load("game_api")
genai.configure(api_key=key)

model = genai.GenerativeModel('gemini-1.5-flash')

In [18]:
# Latest date in the dataset
latest_date = df['InvoiceDate'].max()

# Recency calculation
rfm_recency = df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (latest_date - x.max()).days})
rfm_recency.columns = ['Recency']
rfm_recency.reset_index(inplace=True)

In [19]:
# Frequency calculation
rfm_frequency = df.groupby('CustomerID').agg({'InvoiceNo': 'nunique'})
rfm_frequency.columns = ['Frequency']
rfm_frequency.reset_index(inplace=True)

In [20]:
# Monetary calculation
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
rfm_monetary = df.groupby('CustomerID').agg({'TotalPrice': 'sum'})
rfm_monetary.columns = ['Monetary']
rfm_monetary.reset_index(inplace=True)

In [24]:
# Combine all three metrics
rfm = pd.merge(rfm_recency, rfm_frequency, on='CustomerID')
rfm = pd.merge(rfm, rfm_monetary, on='CustomerID')
# Display the dataframe
display(rfm)

,CustomerID,Recency,Frequency,Monetary
0,12346.0,325,2,0.00
1,12347.0,1,7,4310.00
2,12348.0,74,4,1797.24
3,12349.0,18,1,1757.55
4,12350.0,309,1,334.40
...,...,...,...,...
4367,18280.0,277,1,180.60
4368,18281.0,180,1,80.82
4369,18282.0,7,3,176.60
4370,18283.0,3,16,2094.88


In [25]:
# Scoring the Recency, Frequency, and Monetary metrics
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])

# Combine scores to create RFM score
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Define segments
rfm['Segment'] = 'Low-Value'
rfm.loc[rfm['RFM_Score'].isin(['111', '112', '113', '121', '122']), 'Segment'] = 'High-Value'
rfm.loc[rfm['RFM_Score'].isin(['211', '212', '213', '221', '222']), 'Segment'] = 'Loyal'
rfm.loc[rfm['RFM_Score'].isin(['311', '312', '313', '321', '322']), 'Segment'] = 'New'


In [26]:
personal_columns = ['CustomerID', 'Country']

In [27]:
import hashlib

def anonymize_data(column):
    return column.apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())

# Apply anonymization
df['CustomerID_Anonymized'] = anonymize_data(df['CustomerID'])
df['Country_Anonymized'] = anonymize_data(df['Country'])

In [28]:
# Create 'TotalPrice' column
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Aggregate data on a monthly basis
df.set_index('InvoiceDate', inplace=True)
monthly_sales = df.resample('M').agg({'TotalPrice': 'sum', 'Quantity': 'sum'}).reset_index()

C:\Users\kyana\AppData\Local\Temp\ipykernel_16532\3982104767.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df.resample('M').agg({'TotalPrice': 'sum', 'Quantity': 'sum'}).reset_index()


In [34]:
query = f"Analyze monthly sales to this data: {monthly_sales} trends and identify any seasonality in the dataset. Provide insights into sales trends and forecast sales for the next quarter."

response = model.generate_content(query)
print(response.text)

## Analyzing Monthly Sales Data

The provided data shows monthly sales for a year. Let's analyze the trends and seasonality:

**1.  Visualizing the Data:**

* **Time Series Plot:** Create a line plot of TotalPrice against InvoiceDate. This will visually demonstrate the trend over time.

* **Seasonal Decomposition:** Decompose the time series into trend, seasonality, and residual components. This can be done using libraries like statsmodels in Python.

**2.  Identifying Trends:**

* **Trend:** The plot shows a general upward trend in sales, with occasional fluctuations. The trend appears to be **non-linear** and may be exponential.

* **Seasonality:** There appears to be a strong seasonal pattern, with higher sales in the **holiday season (November-December)** and a dip in sales in the **first quarter (January-March)**.

**3.  Analyzing the Simple Moving Average (SMA):**

* **SMA:** The SMA is a measure of the average sales over a specific period (likely 3 months in this case). It smoot

In [30]:
# Calculate Simple Moving Average (SMA) for the next quarter
monthly_sales['SMA'] = monthly_sales['TotalPrice'].rolling(window=3).mean()

# Forecast for the next quarter
next_quarter_forecast = monthly_sales['SMA'].iloc[-1]

In [31]:
next_quarter_forecast

988709.6433333332